# CNN on CIFAR10

In [1]:
import torch
import torchvision
import torchsummary

## General Constants

In [2]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
N_EPOCHS = 30
PATH = '.cifar_model.pth'

## Loading the data

In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


## The Model

In [4]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=5,
            padding='same',
            stride=1,
        )
        self.relu = torch.nn.ReLU()
        self.max_pool = torch.nn.MaxPool2d(2)
        self.flatten = torch.nn.Flatten()
        self.dense = torch.nn.Linear(
            in_features=8192,
            out_features=10
        )
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)
        return x

In [5]:
model = Model()

torchsummary.summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           2,432
              ReLU-2           [-1, 32, 32, 32]               0
         MaxPool2d-3           [-1, 32, 16, 16]               0
           Flatten-4                 [-1, 8192]               0
            Linear-5                   [-1, 10]          81,930
           Softmax-6                   [-1, 10]               0
Total params: 84,362
Trainable params: 84,362
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.63
Params size (MB): 0.32
Estimated Total Size (MB): 0.96
----------------------------------------------------------------


In [6]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [7]:
for epoch in range(N_EPOCHS):
    training_cost = 0.0
    training_correct = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        predictions = model(inputs)
        cost = loss_func(predictions, targets)
        cost.backward()
        optimizer.step()
        training_cost += cost.item()
        training_correct += (torch.argmax(predictions, dim=1) == targets).sum()
    training_cost /= len(train_set)
    training_correct /= len(train_set)
    print(f'epoch: {epoch}, loss: {training_cost}, accuracy: {training_correct}')

epoch: 0, loss: 0.015904964723587035, accuracy: 0.4272800087928772
epoch: 1, loss: 0.015096047685146332, accuracy: 0.5322999954223633
epoch: 2, loss: 0.014794114100933075, accuracy: 0.5718200206756592
epoch: 3, loss: 0.014603384749889373, accuracy: 0.5968199968338013
epoch: 4, loss: 0.014413966915607453, accuracy: 0.621940016746521
epoch: 5, loss: 0.014277198133468628, accuracy: 0.6399199962615967
epoch: 6, loss: 0.014153855376243592, accuracy: 0.6564000248908997
epoch: 7, loss: 0.014065924696922302, accuracy: 0.667959988117218
epoch: 8, loss: 0.013979237813949585, accuracy: 0.6791800260543823
epoch: 9, loss: 0.013890659244060516, accuracy: 0.6909400224685669
epoch: 10, loss: 0.013830155682563782, accuracy: 0.6980599761009216
epoch: 11, loss: 0.013767866780757905, accuracy: 0.7066199779510498
epoch: 12, loss: 0.013704174511432647, accuracy: 0.7148799896240234
epoch: 13, loss: 0.013653777306079865, accuracy: 0.722000002861023
epoch: 14, loss: 0.013608135764598846, accuracy: 0.7275199890

## Save the trained model

In [8]:
torch.save(model.state_dict(), PATH)

## Load the trained model

In [9]:
model = Model()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Testing the model

In [10]:
correct = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        predictions = model(inputs)
        torch.argmax(predictions)
        correct += (torch.argmax(predictions, dim=1) == targets).sum()

print(f'Accuracy on training set: {correct / len(test_set)}')

Accuracy on training set: 0.659500002861023
